In [500]:
import numpy as np
import random
import copy

## [MUST RUN] HexBoard skeleton function
Same as vanilla skeleton except following addition:
- Freddy has added .get_allempty()

In [561]:
# Hex board skeleton
# commented, no edit

class HexBoard:
    BLUE = 1 # value take up by a position
    RED = 2
    EMPTY = 3

    def __init__(self, board_size): # constructor, set board size
        self.board = {}
        self.size = board_size
        self.game_over = False # state of game over
        for x in range(board_size):
            for y in range (board_size):
                self.board[x,y] = HexBoard.EMPTY
        
    def is_game_over(self): # check if it's game over
        return self.game_over

    def is_empty(self, coordinates): # check if position is empty
        return self.board[coordinates] == HexBoard.EMPTY

    def is_color(self, coordinates, color): # check if position contain certain color 1/2
        return self.board[coordinates] == color

    def get_color(self, coordinates): # read color of a position
        if coordinates == (-1,-1):
            return HexBoard.EMPTY
        return self.board[coordinates]

    def place(self, coordinates, color): # place a piece of color at a position, make move? will update game over state
        if not self.game_over and self.board[coordinates] == HexBoard.EMPTY: # check condition if it's not game over AND position is empty
            self.board[coordinates] = color # update the color 
            if self.check_win(HexBoard.RED) or self.check_win(HexBoard.BLUE): # check win for either color
                self.game_over = True # if check win is true, one side win, then update game over state to true
        
    def get_opposite_color(self, current_color): # return opposite color. what is the purpose?
        if current_color == HexBoard.BLUE:
            return HexBoard.RED
        return HexBoard.BLUE

    def get_neighbors(self, coordinates): # return a list of valid neigbor coordinates from a position
        (cx,cy) = coordinates
        neighbors = []
        if cx-1>=0:   neighbors.append((cx-1,cy))
        if cx+1<self.size: neighbors.append((cx+1,cy))
        if cx-1>=0    and cy+1<=self.size-1: neighbors.append((cx-1,cy+1))
        if cx+1<self.size  and cy-1>=0: neighbors.append((cx+1,cy-1))
        if cy+1<self.size: neighbors.append((cx,cy+1))
        if cy-1>=0:   neighbors.append((cx,cy-1))
        return neighbors

    def border(self, color, move): # check if a move is the right color reaching the right border, blue1-x, red2-y
        (nx, ny) = move
        return (color == HexBoard.BLUE and nx == self.size-1) or (color == HexBoard.RED and ny == self.size-1)

    def traverse(self, color, move, visited): # move is the target position 
        if not self.is_color(move, color) or (move in visited and visited[move]): return False # check if move position do NOT contain my color AND is NOT in visited
        if self.border(color, move): return True # check if the move is reaching border
        visited[move] = True # update position in visited (move history)
        for n in self.get_neighbors(move): # check all neigbour positions if the move passes all checks above
            if self.traverse(color, n, visited): return True
        return False

    def check_win(self, color): # check win condition
        for i in range(self.size):
            if color == HexBoard.BLUE: move = (0,i) # for blue, move rightward (0,0), (0,1), (0,2), ... = start check from one border
            else: move = (i,0) # for red, move downward (0,0), (1,0), (2,0), ...
            if self.traverse(color, move, {}): # if true in traverse, return win. note that traverse will return check if right color reach the right border
                return True
        return False

    def print(self):
        print("   ",end="")
        for y in range(self.size):
            print(chr(y+ord('a')),"",end="") # print x axis id
        print("")
        print(" -----------------------")
        for y in range(self.size):
            print(y, "|",end="") # print y axis id
            for z in range(y):
                print(" ", end="") # print space
            for x in range(self.size):
                piece = self.board[x,y] # read position
                if piece == HexBoard.BLUE: print("b ",end="") # end=print without newline
                elif piece == HexBoard.RED: print("r ",end="")
                else:
                    if x==self.size:
                        print("-",end="")
                    else:
                        print("- ",end="")
            print("|") # print '|' and new line by default
        print("   -----------------------")

    # return list of empty positions
    def get_allempty(self):
        '''Return a list of empty positions in current board, same as movelist.'''
        return [k for k, v in self.board.items() if v == 3] # 3 = EMPTY

## [MUST RUN] Search function - Minimax algorithm, depth limited

In [559]:
# search function - minimax, depth-limited
# dependencies
import numpy as np
import copy
import random

def minimax(board, depth, ntype, p):
    '''
    minimax function, depth-limited
    Parameters: 
        board (HexBoard object): 
        depth (int): depth limit of search tree, if depth exceeds empty positions, it will be reduced
        ntype (str): node type, etiher 'MAX' or 'MIN'
        p (int): perspective/player of search tree root, either 1 for HexBoard.BLUE, or 2 for HexBoard.RED
    Ouputs: 
        node (dict): {'state', 'depth', 'children', 'type', 'score', 'move'}
    Further improvements:
        search statistics: nodes searched + cutoffs
    '''
    # movelist for current state
    movelist = board.get_allempty()

    # for small board and end game, depth limit == full depth
    if depth > len(movelist):
        print('WARNING: DEPTH is limited by empty positions in board => set to full depth search.\n')
        depth = len(movelist)
    
    # initialize node
    n = {'state': board,
         'depth': depth,
         'children': {},
         'type': ntype}
    
    # print to eyetest
    print('\nNode DEPTH = {} (TYPE = {})'.format(n['depth'], n['type']))
    print(' GAME OVER?', n['state'].game_over)
    if (depth != 0) and not (n['state'].is_game_over()):
        print(' PLAYER {} to consider EMPTY positions {}'.format(p, movelist))
    
    # initialize child_count to count children at depth d
    child_count = 0 
    
    # main loop
    if n['state'].is_game_over(): # case: gameover at depth >= 0 (do we need to give bonus or penalty to score?)
        n['type'] = 'LEAF_ENDGAME'
        n['score'] = random.sample(range(0, 10), 1)[0]
        #n['score'] = eval(n['state'])
        print(' Leaf SCORE (ENDGAME) =', n['score'], '\n')
        return n
    
    elif depth == 0: # case: reaching the search tree depth limit
        n['type'] = 'LEAF_HEURISTIC'
        n['score'] = random.sample(range(0, 10), 1)[0]
        #n['score'] = eval(n['state'])
        print(' Leaf SCORE (HEURISTIC) =', n['score'], '\n')
        return n
    
    elif n['type'] == 'MAX': # max node
        g_max = -np.inf # initialize max score with very small
        n['score'] = g_max
        for child_move in movelist: # search all children and compare score
            child_count = child_count+1
            print('\nFrom DEPTH {} branch --> Child {}: \nPLAYER {} moves at {}'.format(n['depth'], child_count, p, child_move))
            print(' STATE before move:')
            n['state'].print()
            new_state = copy.deepcopy(n['state']) # copy state to aviod modifying current state
            new_state.place(child_move, p) # generate child state
            print(' STATE after move:')
            new_state.print() # eyetest child state
            new_p = [1, 2]
            new_p.remove(p) # reverse persective for child node
            child_n = minimax(new_state, n['depth']-1, 'MIN', new_p[0]) # generate child node
            n['children'].update({str(child_move): child_n}) # store children node
            if child_n['score'] > g_max: # update current node to back up from the maximum child node
                g_max = child_n['score']
                n['score'] = child_n['score']
                n['move'] = child_move
            print('End of PLAYER {} DEPTH {} {} node: Child move {} score = {}; Updated optimal move {} score = {}.'.format(p, n['depth'], n['type'], child_move, child_n['score'], n['move'], n['score']))
            
    elif n['type'] == 'MIN': # min node
        g_min = np.inf # initialize min score with very large
        n['score'] = g_min
        for child_move in movelist:
            child_count = child_count+1
            print('\nFrom DEPTH {} branch --> Child {}: \nPLAYER {} moves at {}'.format(n['depth'], child_count, p, child_move))
            print(' STATE before move:')
            n['state'].print()
            new_state = copy.deepcopy(n['state'])
            new_state.place(child_move, p) # generate child state
            print(' STATE after move:')
            new_state.print()
            new_p = [1, 2]
            new_p.remove(p) # reverse persective for child node
            child_n = minimax(new_state, n['depth']-1, 'MAX', new_p[0])
            n['children'].update({str(child_move): child_n}) # store children node
            if child_n['score'] < g_min: # update current node to back up from the minimum child node
                g_min = child_n['score']
                n['score'] = child_n['score']
                n['move'] = child_move
            print('End of PLAYER {} DEPTH {} {} node: Child move {} score = {}; Updated optimal move {} score = {}.'.format(p, n['depth'], n['type'], child_move, child_n['score'], n['move'], n['score']))
                
    else: 
        error('Error: Nothing to execute.')
        return

    return n # g is the maximun heuristic function value
        

#### [outstanding] evaluation function is required

In [ ]:
# evaluation function


#### [WIP] function to print score to visualize result easily

In [555]:
def printChildrenScore(n):
    '''
    Print all children names and scores. Recursive.
    Parameter:
        n: search tree node
    Return:
        None
    '''
    print('depth', n['depth'], 'score', n['score'])
    if n['children'] != {}:
        for k in n['children'].keys():
            print('--> child', k)
            printChildrenScore(n['children'][str(k)])

# Test cells - Demonstration of minimax()

Test plan for the minimax()
- enumerate full width of search tree
- correct value back up mechanism

## Using board size = 2, No. of empty position = 3, Who's turn = RED

In [556]:
# set up
board = HexBoard(2)
# place
board.place((1,1), 1) # BLUE first
board.print()

   a b 
 -----------------------
0 |- - |
1 | - b |
   -----------------------


In [557]:
# now it's RED turn
# use search function!
# output is the search tree root node, dict type
result_node = minimax(board, 3, 'MAX', 2)


Node DEPTH = 3 (TYPE = MAX)
 GAME OVER? False
 PLAYER 2 to consider EMPTY positions [(0, 0), (0, 1), (1, 0)]

From DEPTH 3 branch --> Child 1: 
PLAYER 2 moves at (0, 0)
 STATE before move:
   a b 
 -----------------------
0 |- - |
1 | - b |
   -----------------------
 STATE after move:
   a b 
 -----------------------
0 |r - |
1 | - b |
   -----------------------

Node DEPTH = 2 (TYPE = MIN)
 GAME OVER? False
 PLAYER 1 to consider EMPTY positions [(0, 1), (1, 0)]

From DEPTH 2 branch --> Child 1: 
PLAYER 1 moves at (0, 1)
 STATE before move:
   a b 
 -----------------------
0 |r - |
1 | - b |
   -----------------------
 STATE after move:
   a b 
 -----------------------
0 |r - |
1 | b b |
   -----------------------

Node DEPTH = 1 (TYPE = MAX)
 GAME OVER? True
 Leaf SCORE (ENDGAME) = 5 

End of PLAYER 1 DEPTH 2 MIN node: child move (0, 1) score = 5. Updated optimal move (0, 1) score = 5.

From DEPTH 2 branch --> Child 2: 
PLAYER 1 moves at (1, 0)
 STATE before move:
   a b 
 ---------

In [477]:
# check root node output
print('optimal move', result_node['move'], 'max score', result_node['score'])

optimal move (1, 0) max score 4


In [480]:
# check scores of first layer of children
for k in result_node['children'].keys():
    print('move', k, 'node score', result_node['children'][str(k)]['score'])

move (0, 0) node score 2
move (0, 1) node score 1
move (1, 0) node score 4


## Using board size = 2, No. of empty position = 3, Who's turn = BLUE

In [440]:
# set up
board = HexBoard(2)
# place
board.place((0,1), 2) # RED first
board.print()

   a b 
 -----------------------
0 |- - |
1 | r - |
   -----------------------


In [467]:
# now it's BLUE turn
# use search function!
# output is the search tree root node, dict type
result_node = minimax(board, 3, 'MAX', 1)


Node DEPTH = 3 
 Node TYPE = MAX
 Movelist = [(0, 0), (1, 0), (1, 1)]
 Game over = False

From DEPTH 3 branch --> Child  1 : 
PLAYER 1 moves at (0, 0)
 STATE before move:
   a b 
 -----------------------
0 |- - |
1 | r - |
   -----------------------
 STATE after move:
   a b 
 -----------------------
0 |b - |
1 | r - |
   -----------------------

Node DEPTH = 2 
 Node TYPE = MIN
 Movelist = [(1, 0), (1, 1)]
 Game over = False

From DEPTH 2 branch --> Child  1 : 
PLAYER 2 moves at (1, 0)
 STATE before move:
   a b 
 -----------------------
0 |b - |
1 | r - |
   -----------------------
 STATE after move:
   a b 
 -----------------------
0 |b r |
1 | r - |
   -----------------------

Node DEPTH = 1 
 Node TYPE = MAX
 Movelist = [(1, 1)]
 Game over = True
Leaf SCORE = 9  EARLY GAME OVER!

End of MIN child: (1, 0) of depth 2

From DEPTH 2 branch --> Child  2 : 
PLAYER 2 moves at (1, 1)
 STATE before move:
   a b 
 -----------------------
0 |b - |
1 | r - |
   -----------------------
 STATE

## Using board size = 3, No. of empty position = 4, Who's turn = BLUE

In [510]:
# set up
board = HexBoard(3)
# place
board.place((1,1), 2) # RED first
board.place((1,0), 1)
board.place((2,1), 2)
board.place((2,0), 1)
board.place((0,2), 2)
board.print()

   a b c 
 -----------------------
0 |- b b |
1 | - r r |
2 |  r - - |
   -----------------------


In [529]:
random.seed(10)
# now it's BLUE turn
# use search function!
# output is the search tree root node, dict type
result_node = minimax(board=board, depth=3, ntype='MAX', p=1)


Node DEPTH = 3 (TYPE = MAX)
 GAME OVER? False
 PLAYER 1 to consider EMPTY positions [(0, 0), (0, 1), (1, 2), (2, 2)]

From DEPTH 3 branch --> Child 1: 
PLAYER 1 moves at (0, 0)
 STATE before move:
   a b c 
 -----------------------
0 |- b b |
1 | - r r |
2 |  r - - |
   -----------------------
 STATE after move:
   a b c 
 -----------------------
0 |b b b |
1 | - r r |
2 |  r - - |
   -----------------------

Node DEPTH = 2 (TYPE = MIN)
 GAME OVER? True
 Leaf SCORE (ENDGAME) = 9 

End of PLAYER 1 DEPTH 3 MAX node: child move (0, 0) score = 9. Updated optimal move (0, 0) score = 9.

From DEPTH 3 branch --> Child 2: 
PLAYER 1 moves at (0, 1)
 STATE before move:
   a b c 
 -----------------------
0 |- b b |
1 | - r r |
2 |  r - - |
   -----------------------
 STATE after move:
   a b c 
 -----------------------
0 |- b b |
1 | b r r |
2 |  r - - |
   -----------------------

Node DEPTH = 2 (TYPE = MIN)
 GAME OVER? True
 Leaf SCORE (ENDGAME) = 0 

End of PLAYER 1 DEPTH 3 MAX node: child mo

In [525]:
# check root node output
print('optimal move', result_node['move'], 'max score', result_node['score'])

optimal move (0, 0) max score 9


## [WIP] How to visualize trees?

In [554]:
# print depth 3 tree
print('root-d3', result_node['move'], '    score = ', result_node['score'])
for k0 in result_node['children']:
    ntemp = result_node['children'][str(k0)]
    print('\n        d2:', k0, '    score = ', ntemp['score'])
    for k1 in ntemp['children']:
        ntemp2 = ntemp['children'][str(k1)]
        print('                   d1:', k1, '    score = ', ntemp2['score'])
        for k2 in ntemp2['children']:
            ntemp3 = ntemp2['children'][str(k2)]
            print('                                              d0:', k2, '    score = ', ntemp3['score'])

root-d3 (0, 0)     score =  9

        d2: (0, 0)     score =  9

        d2: (0, 1)     score =  0

        d2: (1, 2)     score =  7
                   d1: (0, 0)     score =  7
                                              d0: (0, 1)     score =  6
                                              d0: (2, 2)     score =  7
                   d1: (0, 1)     score =  9
                                              d0: (0, 0)     score =  9
                                              d0: (2, 2)     score =  0
                   d1: (2, 2)     score =  7
                                              d0: (0, 0)     score =  3
                                              d0: (0, 1)     score =  7

        d2: (2, 2)     score =  2
                   d1: (0, 0)     score =  7
                                              d0: (0, 1)     score =  7
                                              d0: (1, 2)     score =  4
                   d1: (0, 1)     score =  2
                             

# Test cells - Experiments on HexBoard class (Junk)

In [61]:
winner = HexBoard.RED
loser = HexBoard.BLUE

board = HexBoard(3)

board.place((1,1), loser)
board.print()
print(board.check_win(loser))
board.place((2,1), loser)
board.print()
print(board.check_win(loser))
board.place((0,1), loser)
board.print()
print(board.check_win(loser))

   a b c 
 -----------------------
0 |- - - |
1 | - b - |
2 |  - - - |
   -----------------------
False
   a b c 
 -----------------------
0 |- - - |
1 | - b b |
2 |  - - - |
   -----------------------
False
   a b c 
 -----------------------
0 |- - - |
1 | b b b |
2 |  - - - |
   -----------------------
True


In [234]:
HexBoard.RED

2

In [63]:
board.board

{(0, 0): 3,
 (0, 1): 1,
 (0, 2): 3,
 (1, 0): 3,
 (1, 1): 1,
 (1, 2): 3,
 (2, 0): 3,
 (2, 1): 1,
 (2, 2): 3}

In [64]:
board.board.values()

dict_values([3, 1, 3, 3, 1, 3, 3, 1, 3])

In [65]:
[k for k, v in board.board.items() if v == 3] # get empty positions

[(0, 0), (0, 2), (1, 0), (1, 2), (2, 0), (2, 2)]

In [68]:
moves = board.get_allempty()
print(moves)

[(0, 0), (0, 2), (1, 0), (1, 2), (2, 0), (2, 2)]


In [53]:
moves.append((0,0))

In [31]:
board.board.keys()

dict_keys([(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)])

In [32]:
set(board.board.keys())

{(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}

In [25]:
board.game_over

True

In [19]:
winner = HexBoard.RED
loser = HexBoard.BLUE

board = HexBoard(3)

board.place((0,0), loser)
board.print()
print(board.check_win(loser))
board.place((0,1), loser)
board.print()
print(board.check_win(loser))
board.place((0,2), loser)
board.print()
print(board.check_win(loser))

   a b c 
 -----------------------
0 |b - - |
1 | - - - |
2 |  - - - |
   -----------------------
False
   a b c 
 -----------------------
0 |b - - |
1 | b - - |
2 |  - - - |
   -----------------------
False
   a b c 
 -----------------------
0 |b - - |
1 | b - - |
2 |  b - - |
   -----------------------
False


In [61]:
# skeleton usage

import numpy as np
#from hex_skeleton import HexBoard

# sanity check that wins are detected
for i in range(0,2): # loop 0,1 so that winner = red AND blue in each run
  winner = HexBoard.RED if i == 0 else HexBoard.BLUE # name of the player
  loser = HexBoard.BLUE if i == 0 else HexBoard.RED
  board = HexBoard(3) # set up board 
  board.place((1,1), loser) # place a piece
  board.place((2,1), loser)
  board.place((1,2), loser)
  board.place((2,2), loser)
  board.place((0,0), winner)
  board.place((1,0), winner)
  board.place((2,0), winner)
  board.place((0,1), winner)
  board.place((0,2), winner)
  assert(board.check_win(winner) == True)
  assert(board.check_win(loser) == False)
  board.print()


   a b c 
 -----------------------
0 |r r r |
1 | r b b |
2 |  r b b |
   -----------------------
   a b c 
 -----------------------
0 |b b b |
1 | - r r |
2 |  - r r |
   -----------------------


In [ ]:
# sanity check that random play will at some point end the game
endable_board = HexBoard(4)
while not endable_board.game_over:
  endable_board.place((np.random.randint(0, 4), np.random.randint(0, 4)), HexBoard.RED)
assert(endable_board.game_over == True)
assert(endable_board.check_win(HexBoard.RED) == True)
assert(endable_board.check_win(HexBoard.BLUE) == False)
print("Randomly filled board")
endable_board.print()

In [ ]:
neighbor_check = HexBoard(5)
assert(neighbor_check.get_neighbors((0, 0)) == [(1, 0), (0, 1)])
assert(neighbor_check.get_neighbors((0, 1)) == [(1, 1), (1, 0), (0, 2), (0, 0)])
assert(neighbor_check.get_neighbors((1, 1)) == [(0, 1), (2, 1), (0, 2), (2, 0), (1, 2), (1, 0)])
assert(neighbor_check.get_neighbors((3, 4)) == [(2, 4), (4, 4), (4, 3), (3, 3)])
assert(neighbor_check.get_neighbors((4, 3)) == [(3, 3), (3, 4), (4, 4), (4, 2)])
assert(neighbor_check.get_neighbors((4, 4)) == [(3, 4), (4, 3)])
neighbor_check_11 = HexBoard(5)
assert(neighbor_check_11.get_neighbors((4, 4)) == [(3, 4), (4, 3)])

In [ ]:
neighbor_check_small = HexBoard(2)
assert(neighbor_check_small.get_neighbors((0, 0)) == [(1, 0), (0, 1)])
assert(neighbor_check_small.get_neighbors((1, 0)) == [(0, 0), (0, 1), (1, 1)])
assert(neighbor_check_small.get_neighbors((0, 1)) == [(1, 1), (1, 0), (0, 0)])
assert(neighbor_check_small.get_neighbors((1, 1)) == [(0, 1), (1, 0)])

In [ ]:
neighbor_check_sanity = HexBoard(11)
for x in range(0, 11):
  for y in range(0, 11):
    neighbors = neighbor_check_sanity.get_neighbors((x, y))
    for neighbor in neighbors:
      neighbors_neighbors = neighbor_check_sanity.get_neighbors(neighbor)
      index_of_self = neighbors_neighbors.index((x, y))
      assert(index_of_self != -1)

In [64]:
board = HexBoard(3)
winner = HexBoard.RED
loser = HexBoard.BLUE

board.place((0,1), winner)
board.place((2,2,), loser)

board.print()
print(board.board[0,0], board.board[0,1], board.board[0,2])
print(board.board[1,0], board.board[1,1], board.board[1,2])
print(board.board[2,0], board.board[2,1], board.board[2,2])

   a b c 
 -----------------------
0 |- - - |
1 | r - - |
2 |  - - b |
   -----------------------
3 2 3
3 3 3
3 3 1


In [416]:
# the game does not allow you to .place() after game over --> early end of some search
tempboard = HexBoard(3)

tempboard.place((0,0), 1)
tempboard.print()
print(tempboard.game_over)

tempboard.place((1,0), 1)
tempboard.print()
print(tempboard.game_over)

tempboard.place((2,0), 1)
tempboard.print()
print(tempboard.game_over)

# can I place after winning
tempboard.place((2,2), 1)
tempboard.print()
print(tempboard.game_over)

   a b c 
 -----------------------
0 |b - - |
1 | - - - |
2 |  - - - |
   -----------------------
False
   a b c 
 -----------------------
0 |b b - |
1 | - - - |
2 |  - - - |
   -----------------------
False
   a b c 
 -----------------------
0 |b b b |
1 | - - - |
2 |  - - - |
   -----------------------
True
   a b c 
 -----------------------
0 |b b b |
1 | - - - |
2 |  - - - |
   -----------------------
True
